# 3. Propiedades estadísticas del lenguaje natural

## Objetivo

- Explorar propiedades estadísticas del lenguaje natural
- Observar si se cumplen propiedades como:
    - La distribución de Zipf
    - La distribución de Heap
- Observar como impacta la normalización 

## Perspectivas formales

- Fueron el primer acercamiento al procesamiento del lenguaje natural. Sin embargo tienen varias **desventajas**
- Requieren **conocimiento previo de la lengua**
- Las herramientas son especificas de la lengua
- Los fenomenos que se presentan son muy amplios y dificilmente se pueden abarcar con reglas formales (muchos casos especiales)
- Las reglas tienden a ser rigidas y no admiten incertidumbre en el resultado

## Perspectiva estadística

- Puede integrar aspectos de la perspectiva formal
- Lidia mejor con la incertidumbre y es menos rigida que la perspectiva formal
- No requiere conocimiento profundo de la lengua. Se pueden obtener soluciones de forma no supervisada

### Modelos estadísticos

- Las **frecuencias** juegan un papel fundamental para hacer una descripción acertada del lenguaje
- Las frecuencias nos dan información de la **distribución de tokens**, de la cual podemos estimar probabilidades.
- Existen **leyes empíricas del lenguaje** que nos indican como se comportan las lenguas a niveles estadísticos
- A partir de estas leyes y otras reglas estadísticas podemos crear **modelos del lenguaje**; es decir, asignar probabilidades a las unidades lingüísticas

In [ ]:
# Bibliotecas
from collections import Counter
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 6]
from re import sub
import numpy as np
import pandas as pd
from itertools import chain

In [ ]:
mini_corpus = """Humanismo es un concepto polisémico que se aplica tanto al estudio de las letras humanas, los
estudios clásicos y la filología grecorromana como a una genérica doctrina o actitud vital que
concibe de forma integrada los valores humanos. Por otro lado, también se denomina humanis-
mo al «sistema de creencias centrado en el principio de que las necesidades de la sensibilidad
y de la inteligencia humana pueden satisfacerse sin tener que aceptar la existencia de Dios
y la predicación de las religiones», lo que se aproxima al laicismo o a posturas secularistas.
Se aplica como denominación a distintas corrientes filosóficas, aunque de forma particular,
al humanismo renacentista1 (la corriente cultural europea desarrollada de forma paralela al
Renacimiento a partir de sus orígenes en la Italia del siglo XV), caracterizado a la vez por su
vocación filológica clásica y por su antropocentrismo frente al teocentrismo medieval
"""
words = mini_corpus.replace("\n", " ").split(" ")
len(words)

In [ ]:
vocabulary = Counter(words)
vocabulary.most_common(10)

In [ ]:
len(vocabulary)

In [ ]:
def get_frequencies(vocabulary: Counter, n: int) -> list:
    return [_[1] for _ in vocabulary.most_common(n)]

def plot_frequencies(frequencies: list, title="Freq of words", log_scale=False):
    x = list(range(1, len(frequencies)+1))
    plt.plot(x, frequencies, "-v")
    plt.xlabel("Freq rank (r)")
    plt.ylabel("Freq (f)")
    if log_scale:
        plt.xscale("log")
        plt.yscale("log")
    plt.title(title)

In [ ]:
frequencies = get_frequencies(vocabulary, 100)
plot_frequencies(frequencies)

In [ ]:
plot_frequencies(frequencies, log_scale=True)

### ¿Qué pasará con más datos? 📊

## Ley Zipf

Exploraremos el Corpus de Referencia del Español Actual [CREA](https://www.rae.es/banco-de-datos/crea/crea-escrito)

<center><img src="img/crea.png"></center>

In [ ]:
!head corpora/zipf/crea_full.csv

In [ ]:
corpus_freqs = pd.read_csv('corpora/zipf/crea_full.csv', delimiter='\t', encoding="latin-1")
N = len(set(chain(*[list(str(w)) for w in corpus_freqs['words'].to_list()])))
corpus_freqs.head(10)

In [ ]:
corpus_freqs.iloc[-10]

In [ ]:
corpus_freqs[corpus_freqs["words"] == "barriga"]

In [ ]:
corpus_freqs["freq"].plot(marker="o")
plt.title('Ley de Zipf en el CREA')
plt.xlabel('rank')
plt.ylabel('freq')
plt.show()

In [ ]:
corpus_freqs['freq'].plot(loglog=True, legend=False)
plt.title('Ley de Zipf en el CREA (log-log)')
plt.xlabel('log rank')
plt.ylabel('log frecuencia')
plt.show()

### Ley de Zipf

- Notamos que las frecuencias entre lenguas siguen un patrón
- Pocas palabras (tipos) son muy frecuentes, mientras que la mayoría de palabras ocurren pocas veces

De hecho, la frecuencia de la palabra que ocupa la posición r en el rank, es proporcional a $\frac{1}{r}$ (La palabra más frecuente ocurrirá aproximadamente el doble de veces que la segunda palabra más frecuente en el corpus y tres veces más que la tercer palabra más frecuente del corpus, etc)

$$f(w_r) \propto \frac{1}{r^α}$$

Donde:
- $r$ es el rank que ocupa la palabra en el corpus
- $f(w_r)$ es la frecuencia de la palabra en el corpus
- $\alpha$ es un parámetro, el valor dependerá del corpus o fenómeno que estemos observando

### Formulación de la Ley de Zipf:

$f(w_{r})=\frac{c}{r^{\alpha }}$

En la escala logarítimica:

$log(f(w_{r}))=log(\frac{c}{r^{\alpha }})$

$log(f(w_{r}))=log (c)-\alpha log (r)$

### ❓ ¿Cómo estimar el parámetro $\alpha$?

In [ ]:
from scipy.optimize import minimize

def calculate_alpha(ranks: np.array, frecs: np.array) -> float:
    # Inicialización
    a0 = 1
    # Función de minimización:
    func = lambda a: sum((np.log(frecs)-(np.log(frecs[0])-a*np.log(ranks)))**2)
    # Minimización: Usando minimize de scipy.optimize:
    return minimize(func, a0).x[0] 

ranks = np.array(corpus_freqs.index) + 1
frecs = np.array(corpus_freqs['freq'])

a_hat = calculate_alpha(ranks, frecs)

print('alpha:', a_hat, '\nMSE:', func(a_hat))

In [ ]:
def plot_generate_zipf(alpha: np.float64, ranks: np.array, freqs: np.array) -> None:
    plt.plot(np.log(ranks), -a_hat*np.log(ranks) + np.log(frecs[0]), color='r', label='Aproximación Zipf')

In [ ]:
#plt.plot(np.log(ranks), -a_hat*np.log(ranks) + np.log(frecs[0]), color='r', label='Aproximación Zipf')
plot_generate_zipf(a_hat, ranks, frecs)
plt.plot(np.log(ranks), np.log(frecs), color='b', label='Distribución original')
plt.xlabel('log ranks')
plt.ylabel('log frecs')
plt.legend(bbox_to_anchor=(1, 1))
plt.show()

## Ley de Heap

Relación entre el número de **tokens** y **tipos** de un corpus

$$T \propto N^b$$

Dónde:

- $T = $ número de tipos
- $N = $ número de tokens
- $b = $ parámetro  

- **TOKENS**: Número total de palabras dentro del texto (incluidas repeticiones)
- **TIPOS**: Número total de palabras únicas en el texto

In [ ]:
# Obtenemos los tipos y tokens
total_tokens = corpus_freqs["freq"].sum()
total_types = len(corpus_freqs)

In [ ]:
# Ordenamos el corpus por frecuencia
corpus_freqs_sorted = corpus_freqs.sort_values(by='freq', ascending=False)

# Calculamos la frecuencia acumulada
corpus_freqs_sorted['cum_tokens'] = corpus_freqs_sorted['freq'].cumsum()

# Calculamos el número acumulado de tipos
corpus_freqs_sorted['cum_types'] = range(1, total_types + 1)

In [ ]:
corpus_freqs_sorted.head()

In [ ]:
# Plot de la ley de Heap
plt.plot(corpus_freqs_sorted['cum_types'], corpus_freqs_sorted['cum_tokens'])
plt.xscale("log")
plt.yscale("log")
plt.xlabel('Types')
plt.ylabel('Tokens')
plt.title('Ley de Heap')
plt.show()

## ¿Otros idiomas? 🇧🇴 🇨🇦 🇲🇽

### Presentando `pyelotl` 🌽

In [ ]:
!pip install elotl

- [Documentación](https://pypi.org/project/elotl/)
- Paquete para desarrollo de herramientas de NLP enfocado en lenguas de bajos recursos digitales habladas en México

In [ ]:
from elotl import corpus as elotl_corpus


print("Name\t\tDescription")
for row in elotl_corpus.list_of_corpus():
    print(row)

Cada corpus se pueden visualizar y navegar a través de interfaz web.
- [Axolotl](https://axolotl-corpus.mx/)
- [Tsunkua](https://tsunkua.elotl.mx/)

In [ ]:
axolotl = elotl_corpus.load("axolotl")
for row in axolotl:
    print("Lang 1 (es) =", row[0])
    print("Lang 2 (nah) =", row[1])
    print("Variante =", row[2])
    print("Documento de origen =", row[3])
    break

In [ ]:
tsunkua = elotl_corpus.load("tsunkua")
for row in tsunkua:
    print("Lang 1 (es) =", row[0])
    print("Lang 2 (oto) =", row[1])
    print("Variante =", row[2])
    print("Documento de origen =", row[3])
    break

In [ ]:
def extract_words_from_sentence(sentence: str) -> list:
    return sub(r'[^\w\s\']', ' ', sentence).lower().split()

def get_words(corpus: list) -> tuple[list, list]:
    words_l1 = []
    words_l2 = []
    for row in corpus:
        words_l1.extend(extract_words_from_sentence(row[0]))
        words_l2.extend(extract_words_from_sentence(row[1]))
    return words_l1, words_l2

In [ ]:
spanish_words_na, nahuatl_words = get_words(axolotl)
spanish_words_oto, otomi_words = get_words(tsunkua)

### Tokens

In [ ]:
print("Número total de palabras en náhuatl (corpus 1):", len(nahuatl_words))
print("Número total de palabras en español (corpus 1):", len(spanish_words_na))
print("Número total de palabras en otomí (corpus 2):", len(otomi_words))
print("Número total de palabras en español (corpus 2):", len(spanish_words_oto))

### ❓ ¿Porqué si son textos paralelos (traducciones) el número de palabras cambia tanto?

De manera general, por las diferencias inherentes de las lenguas para expresar los mismos conceptos, referencias, etc. De manera particular, estas diferencias revelan características morfológicas de las lenguas. El náhuatl es una lengua con tendencia aglutinante/polisintética, por lo tanto, tiene menos palabras pero con morfología rica que les permite codificar cosas que en lenguas como el Español aparecen en la sintaxis. Ejemplo:

> titamaltlakwa - Nosotros comemos tamales

### Tipos

In [ ]:
nahuatl_vocabulary = Counter(nahuatl_words)
nahuatl_es_vocabulary = Counter(spanish_words_na)
otomi_vocabulary = Counter(otomi_words)
otomi_es_vocabulary = Counter(spanish_words_oto)

In [ ]:
otomi_vocabulary.most_common(10)

In [ ]:
print("Tamaño del vocabulario de nahúatl:", len(nahuatl_vocabulary))
print("Tamaño del vocabulario de español (corpus 1):", len(nahuatl_es_vocabulary))
print("Tamaño del vocabulario de otomí:", len(otomi_vocabulary))
print("Tamaño del vocabulario de español (corpus 2):", len(otomi_es_vocabulary))

### ❓ ¿Cómo cambiarían estas estadísticas si no filtramos los signos de puntuación?

Si no normalizamos aumenta el número de tipos lo cual "ensucia" los datos con los que vamos a trabajar. Ejemplo: `algo != algo,`

### ❓ ¿Cómo afecta la falta de normalización ortográfica en lenguas como el náhuatl

En lenguas como el nahúatl, la falta de normalización ortográfica y las variaciones diacrónicas del corpus, provocan que haya grafías diferentes que corresponden a una misma palabra. Ejemplo:

- Yhuan-ihuan

- Yn-in

In [ ]:
print(nahuatl_vocabulary["in"])
print(nahuatl_vocabulary["yn"])

### Normalizador para el Nahúatl

In [ ]:
from elotl.nahuatl import orthography

normalizer = orthography.Normalizer("inali")

In [ ]:
help(normalizer)

In [ ]:
nahuatl_words_normalized = [normalizer.normalize(word) for word in nahuatl_words]

In [ ]:
nahuatl_norm_vocabulary = Counter(nahuatl_words_normalized)
print("Tamaño del vocabulario (tipos) ANTES de normalizar:", len(nahuatl_vocabulary))
print("Tamaño del vocabulario (tipos) DESPUÉS de normalizar:", len(nahuatl_norm_vocabulary))

In [ ]:
def avg_len(tokens: list) -> float:
    return sum(len(token) for token in tokens) / len(tokens)

In [ ]:
print("Longitud promedio de palabras en nahúatl:", avg_len(nahuatl_words))
print("Longitud promedio de palabras en nahúatl (NORM):", avg_len(nahuatl_words_normalized))
print("Longitud promedio de palabras en otomí:", avg_len(otomi_words))
print("Longitud promedio de palabras en español (corpus 1):", avg_len(spanish_words_na))
print("Longitud promedio de palabras en español (corpus 2):", avg_len(spanish_words_oto))

#### Ejercicio: Obtener la palabra más larga de cada lista de palabras (10 min) (0.5 pt extra 🔥)

In [ ]:
# Manda tu solucion en un PR :)